In [1]:
from valuesider_crawler.portfolio_crawler import PortfolioCrawler

%load_ext autoreload
%autoreload 2

In [2]:
crawler = PortfolioCrawler('https://valuesider.com/guru/warren-buffett-berkshire-hathaway/portfolio/2013/2')
d = await crawler.crawl()

2024-11-23 10:34:11 - portfolio_crawler.py:118 - INFO - Crawling quarter: https://valuesider.com/guru/warren-buffett-berkshire-hathaway/portfolio/2013/2


2024-11-23 10:34:13 - portfolio_crawler.py:70 - INFO - Processing page 1 for quarter 2013-06-30: https://valuesider.com/guru/warren-buffett-berkshire-hathaway/portfolio/2013/2
2024-11-23 10:34:18 - portfolio_crawler.py:70 - INFO - Processing page 2 for quarter 2013-06-30: https://valuesider.com/guru/warren-buffett-berkshire-hathaway/portfolio/2013/2?sells_page=1&page=2
2024-11-23 10:34:23 - portfolio_crawler.py:70 - INFO - Processing page 3 for quarter 2013-06-30: https://valuesider.com/guru/warren-buffett-berkshire-hathaway/portfolio/2013/2?sells_page=1&page=3
2024-11-23 10:34:24 - portfolio_crawler.py:85 - INFO - No more pages to process
2024-11-23 10:34:25 - portfolio_crawler.py:118 - INFO - Crawling quarter: https://valuesider.com/guru/warren-buffett-berkshire-hathaway/portfolio/2013/3
2024-11-23 10:34:26 - portfolio_crawler.py:70 - INFO - Processing page 1 for quarter 2013-09-30: https://valuesider.com/guru/warren-buffett-berkshire-hathaway/portfolio/2013/3
2024-11-23 10:34:31 - p

In [10]:
d['2024-09-30'].sort_values('portfolio_weight', ascending=False)

,symbol,company,portfolio_weight,shares,buy_price,current_price,value,change,crawl_timestamp
4,AAPL,APPLE INC,0.2624,300000000.0,233.00,229.87,6.990000e+13,NaN,2024-11-23 10:44:51.555146
2,AXP,AMERICAN EXPRESS CO,0.1544,84901992.0,271.20,301.30,2.302542e+13,NaN,2024-11-23 10:44:51.555146
6,BAC,BANK OF AMERICA CORP,0.1188,434737402.0,39.68,47.00,1.725038e+13,NaN,2024-11-23 10:44:51.555146
12,KO,COCA-COLA CO/THE,0.1079,400000000.0,71.86,63.92,2.874400e+13,NaN,2024-11-23 10:44:51.555146
9,CVX,CHEVRON CORP,0.0656,62863583.0,147.27,162.36,9.257920e+12,NaN,2024-11-23 10:44:51.555146
31,OXY,OCCIDENTAL PETROLEUM CORP,0.0494,239964632.0,51.54,51.93,1.236778e+13,NaN,2024-11-23 10:44:51.555146
28,MCO,MOODY'S CORP,0.0440,24669778.0,474.59,480.66,1.170803e+13,NaN,2024-11-23 10:44:51.555146
18,KHC,KRAFT HEINZ CO/THE,0.0429,325634818.0,35.11,31.81,1.143304e+13,NaN,2024-11-23 10:44:51.555146
10,CB,CHUBB LTD,0.0293,27033784.0,288.39,285.21,7.796273e+12,NaN,2024-11-23 10:44:51.555146
13,DVA,DAVITA INC,0.0222,18589218.0,163.93,165.51,3.047331e+12,NaN,2024-11-23 10:44:51.555146


In [4]:
d.keys()

dict_keys(['2013-06-30', '2013-09-30', '2013-12-31', '2014-03-31', '2014-06-30', '2014-09-30', '2014-12-31', '2015-03-31', '2015-06-30', '2015-09-30', '2015-12-31', '2016-03-31', '2016-06-30', '2016-09-30', '2016-12-31', '2017-03-31', '2017-06-30', '2017-09-30', '2017-12-31', '2018-03-31', '2018-06-30', '2018-09-30', '2018-12-31', '2019-03-31', '2019-06-30', '2019-09-30', '2019-12-31', '2020-03-31', '2020-06-30', '2020-09-30', '2020-12-31', '2021-03-31', '2021-06-30', '2021-09-30', '2021-12-31', '2022-03-31', '2022-06-30', '2022-09-30', '2022-12-31', '2023-03-31', '2023-06-30', '2023-09-30', '2023-12-31', '2024-03-31', '2024-06-30', '2024-09-30'])